In [43]:
import pandas as pd
df = pd.read_csv('original_dataset.csv', low_memory = False, dtype = {'locality': str, 'postcode': str, 'property_subtype': str})
df['locality'] = pd.to_numeric(df['locality'], errors='coerce')
df['postcode'] = pd.to_numeric(df['postcode'], errors='coerce') 
df["locality"].fillna(df["postcode"], inplace=True)
df = df.loc[(pd.isnull(df['postcode'])==False) & (pd.isnull(df['locality'])==False)]
df['property_subtype']=df['property_subtype'].str.upper()
df['property_subtype'] = df['property_subtype'].str.replace('-','_')
df['rooms_number'] = df['rooms_number'].str.replace('None','')
df['rooms_number'] = df['rooms_number'].str.replace('Not specified','')
df['rooms_number'] = df['rooms_number'].str.replace('nan','')
df['rooms_number'] = pd.to_numeric(df['rooms_number'], errors='coerce').astype('Int64')
df.loc[(df['terrace']=='2.0')|(df['terrace']=='3.0')|(df['terrace']=='4.0')]
import numpy as np
def converter(c): 
    if (c['garden']=='True')|(c['garden']=='False'): 
        return c['garden'] 
    elif (pd.isnull(c['garden'])==False): 
        return 'True'
    else: 
        return np.nan

df['hyp_garden'] = df.apply(converter, axis = 1)
def splitter(s): 
    if (s['garden']!='True')&(s['garden']!='False'): 
        return s['garden']
    elif (pd.isnull(s['garden'])==False): 
        return s['garden_area']
    else: 
        return np.nan
    
df['hyp_garden_area'] = df.apply(splitter, axis = 1)
df['hyp_garden_area'] = pd.to_numeric(df['hyp_garden_area'], errors='coerce').astype('Int64')
df.loc[df['facades_number']=='None', 'facades_number'] = np.nan
df['facades_number'] = pd.to_numeric(df['facades_number'], errors='coerce').astype('Int64')
df['swimming_pool_has'] = df['swimming_pool_has'].str.replace('TRUE','True')
df['swimming_pool_has'] = df['swimming_pool_has'].str.replace('FALSE','False')
df['swimming_pool_has'] = df['swimming_pool_has'].str.replace('0','False')
df['building_state'] = df['building_state'].str.replace(' ','_')
df['building_state'] = df['building_state'].str.upper()
df['building_state'] = df['building_state'].str.replace('TO_RESTORE','TO_RENOVATE')
df['building_state'] = df['building_state'].str.replace('TO_BE_DONE_UP','TO_RENOVATE')
df['building_state'] = df['building_state'].str.replace('JUST_RENOVATED','AS_NEW')
df.loc[df['building_state'] == 'NOT_SPECIFIED','building_state'] = np.nan
df.loc[df['building_state'] == '0','building_state'] = np.nan

In [44]:
df.dtypes

source                 int64
hyperlink             object
locality             float64
postcode             float64
house_is                bool
property_subtype      object
price                 object
sale                  object
rooms_number           Int64
area                  object
kitchen_has           object
furnished             object
open_fire             object
terrace               object
terrace_area          object
garden                object
garden_area           object
land_surface          object
land_plot_surface     object
facades_number         Int64
swimming_pool_has     object
building_state        object
hyp_garden            object
hyp_garden_area        Int64
dtype: object

### Remarks & questions

- Pourquoi parfois on applique une formule sur pd et parfois sur df ? 
- Pourquoi la conversion vers boolean ne supporte les nan ? 
- Idée : à partir de maintenant j'aimerais vérifier si mes conversion se passe bien, je vais fonctionner avec colonne intermediaire
- Why does this give a weird result : df['price'].dtypes ? Result is dtype('O')

In [50]:
df['price'] = df['price'].astype(str)

In [ ]:
df['hyp_garden'] = df.apply(converter, axis = 1)
def splitter(s): 
    if (s['garden']!='True')&(s['garden']!='False'): 
        return s['garden']
    elif (pd.isnull(s['garden'])==False): 
        return s['garden_area']
    else: 
        return np.nan
    
df['hyp_garden_area'] = df.apply(splitter, axis = 1)

In [63]:
df[df["my_price"].str.contains('[/€]', regex=True)]


,source,hyperlink,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,...,garden,garden_area,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state,hyp_garden,hyp_garden_area,my_price


In [ ]:
def integer_conversion(s): 
    if (s["price"].contains('[/€]', regex=True)): 
        return s['price']
    else: 
        return "test"
df['my_price'] = df.apply(integer_conversion, axis = 1)

In [60]:
cols_to_change = ['price']

for col in cols_to_change:
    df['my_price'] = df[col].str.replace('[€,]', '')

In [67]:
cols_to_change = ['price']

def any_curr(s, curr="¥$€£"):
    return any(c in s for c in curr)

for col in cols_to_change:
    if df[col].apply(any_curr):
        "bim"
    else: 
        "test"

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [66]:
cols_to_change = ['price']

for col in cols_to_change:
    if df[col].str.contains('[/€]', regex=True):
        "bim"
    else: 
        "test"

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [65]:
df[col].str.contains('[/€]', regex=True)

0        False
1        False
2        False
3        False
4        False
         ...  
93063     True
93064     True
93065     True
93066     True
93067     True
Name: price, Length: 85656, dtype: bool

In [30]:
df['my_price'] = df['price'].astype(str)
#df['price'].astype(str).astype(int) -> gives a bug

In [18]:
df.loc[df['my_price'] == 'False']

,source,hyperlink,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,...,garden,garden_area,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state,hyp_garden,hyp_garden_area,my_price


In [31]:
df = df.loc[(pd.isnull(df['my_price'])==False)]

In [57]:
new_df = df[~df["my_price"].str.contains('[/€]', regex=True)]
new_df
#all these should be entering in a new int column, for those with the symbol it's more work 
#.str.contains does not work for my function (no attribute 'str' or 'contains')

,source,hyperlink,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,...,garden,garden_area,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state,hyp_garden,hyp_garden_area,my_price
0,6,8901695,4180.0,4180.0,True,MIXED_USE_BUILDING,295000,NaN,3,242,...,True,1000,1403,1403,0,False,GOOD,True,1000,test
1,6,8747010,8730.0,8730.0,True,VILLA,675000,NaN,4,349,...,True,977,1526,1526,0,False,AS_NEW,True,977,675000
2,6,8775843,4020.0,4020.0,True,APARTMENT_BLOCK,250000,NaN,5,303,...,False,0,760,760,0,False,TO_RENOVATE,False,0,test
3,6,8910441,1200.0,1200.0,True,HOUSE,545000,NaN,4,235,...,False,0,63,63,0,False,AS_NEW,False,0,test
4,6,8758672,1190.0,1190.0,True,MIXED_USE_BUILDING,500000,NaN,2,220,...,True,60,193,193,0,False,AS_NEW,True,60,test
5,6,8940340,4040.0,4040.0,True,HOUSE,189000,NaN,3,200,...,True,40,100,11,0,False,TO_RENOVATE,True,40,test
6,6,8923626,4540.0,4540.0,True,MIXED_USE_BUILDING,465000,NaN,4,400,...,False,0,312,312,0,False,GOOD,False,0,test
7,6,8913667,1150.0,1150.0,True,APARTMENT_BLOCK,650000,NaN,4,200,...,True,150,301,301,0,False,GOOD,True,150,test
8,6,8693767,6870.0,6870.0,True,MIXED_USE_BUILDING,89000,NaN,3,180,...,False,0,96,96,0,False,TO_RENOVATE,False,0,test
9,6,8325945,4030.0,4030.0,True,MIXED_USE_BUILDING,129000,NaN,3,156,...,False,0,71,71,0,False,TO_RENOVATE,False,0,test
